# Imports

In [ ]:
from tools import *

# Parameters

README 

- `non_track_keys = 5` has to be updated if new keys are added to an event in metrics

In [ ]:
# File paths
path_prefix = "../../data/SoLAr_v2/"
charge_bucket = "cosmics/root/"
light_bucket = "root/46v_12db_th950_deco/"

input = "deco_v3_0cd913fa_20230706_191437.data.root"

In [ ]:
# Load options
params.reload_files = True
params.rematch_events = False

# Save options
params.overwrite_metrics = True
params.save_figures = True

# Plotting options
params.flip_x = True
params.individual_plots = np.arange(1, 11, 1)
params.show_figures = False
params.label_font_size = 16
params.tick_font_size = 16
params.title_font_size = 18

# Events to process
params.event_list = None

# Noisy Pixels
params.channel_disable_list = [(7, (1, 2))]  # (chip, channel)

# Light variable to consider
params.light_variable = "integral"

# Units for plot labels
params.q_unit = "e"  # After applying charge_gain
params.xy_unit = "mm"
params.z_unit = "mm"
params.time_unit = "ns"

# Conversion factors
params.detector_z = 300
params.detector_x = 128
params.detector_y = 160

# DBSCAN parameters for charge clustering
params.min_samples = 2
params.xy_epsilon = 8  # 8 ideal
params.z_epsilon = 8  # 8 ideal

# RANSAC parameters for line fitting
params.ransac_residual_threshold = 6  # 6 ideal for charge, 35 ideal for light
params.ransac_max_trials = 1000
params.ransac_min_samples = 2  # 2 ideal for charge, 3 ideal for light

# Force parameters for cylinder
params.force_dh = None
params.force_dr = None

# Filters for post processing
params.score_cutoff = -1.0
params.max_score = 1.0
params.min_track_length = 30
params.max_track_length = np.inf
params.max_tracks = 1

# Other
params.non_track_keys = 5

In [ ]:
file_label = "_".join(input.split("_")[-2:]).split(".")[0]
charge_timestamp = pd.to_datetime(file_label, format="%Y%m%d_%H%M%S").strftime(
    "%Y_%m_%d_%H_%M"
)

light_file = path_prefix + "Light/" + light_bucket + input
charge_file = (
    path_prefix
    + "Charge/"
    + charge_bucket
    + f"evd_self_trigger-packets-{charge_timestamp}_CEST_validated.root"
)

In [ ]:
recal_params()

## Parameter calculators

# File handing

## Loading

In [ ]:
# If match dictionary already exists for this file label, load it
temp_filename = f"{file_label}/match_dict_{file_label}.json"
if not params.rematch_events and os.path.isfile(temp_filename):
    with open(temp_filename, "r") as f:
        match_dict = json.load(f)
        match_dict = {int(key): value for key, value in match_dict.items()}

    print("Match_dict loaded from file")

del temp_filename

In [ ]:
temp_filename = f"{file_label}/charge_df_{file_label}.bz2"
if not params.reload_files and os.path.isfile(temp_filename):
    charge_df = pd.read_csv(temp_filename, index_col=0)
    if not params.event_list is None:
        charge_df = charge_df.loc[params.event_list.intersection(charge_df.index)]
    # for column in :
    charge_df[charge_df.columns[9:]] = charge_df[charge_df.columns[9:]].applymap(
        lambda x: literal_eval(x) if isinstance(x, str) else x
    )
else:
    charge_df = load_charge(charge_file, events=params.event_list)

del temp_filename

In [ ]:
# Clean up charge dataframe

# Remove events with negative charge hits and without light trigger
charge_mask = (
    charge_df["event_hits_q"].apply(tuple).explode().groupby("eventID").min() > 0
) * (charge_df["trigID"].apply(len) > 0)
charge_df = charge_df[charge_mask]

print(
    f"Removed charge events: {charge_mask.count()-charge_mask.sum()}/{charge_mask.count()}"
)

In [ ]:
# If match dictionary already loaded, match loaded charge events to light events before loading light events.
light_event_list = None
if not match_dict == {}:
    light_event_list = ak.flatten(
        [
            match_dict.get(str(event), [])
            for event in charge_df.index
            if params.event_list is None or event in params.event_list
        ]
    )

    print(f"{len(light_event_list)} light events to load.")

In [ ]:
temp_filename = f"{file_label}/light_df_{file_label}.bz2"
# Load light events using light_event_list if loaded via match dictionary
if not params.reload_files and os.path.isfile(temp_filename):
    light_df = pd.read_csv(temp_filename, index_col=0)
    if not light_event_list is None:
        light_df = light_df[light_df["event"].isin(light_event_list)]

else:
    light_df = load_light(
        light_file, deco="deco" in light_file, events=light_event_list
    )

print(f"{light_df['event'].nunique()} light events loaded.")
del temp_filename

In [ ]:
# Clean up light dataframe

# If match dictionary not yet loaded, create it
if match_dict == {} or params.rematch_events:
    match_dict = match_events(charge_df, light_df)

    # Remove light events without charge event match
    light_events = np.unique(ak.flatten(match_dict.values()))
    light_df = light_df[light_df["event"].isin(light_events)]

    print(
        f"Remaining light events with charge event match: {light_df['event'].nunique()}"
    )

# Remove charge events without associated light event
charge_df = charge_df.loc[match_dict.keys()]

print(f"Remaining charge events with light match: {len(charge_df)}")

In [ ]:
# Flip x axis according to flag

charge_df["event_hits_x"] = charge_df["event_hits_x"].apply(
    lambda x: [np.power(-1, params.flip_x) * i for i in x]
)
light_df["x"] = light_df["x"].apply(lambda x: np.power(-1, params.flip_x) * x)

## Saving

In [ ]:
os.makedirs(file_label, exist_ok=True)

In [ ]:
# Only save files if all events were considered, i.e. event_list is None
if params.event_list is None:
    charge_df.to_csv(f"{file_label}/charge_df_{file_label}.bz2")
    light_df.to_csv(f"{file_label}/light_df_{file_label}.bz2")
    with open(f"{file_label}/match_dict_{file_label}.json", "w") as f:
        json.dump(match_dict, f)

In [ ]:
# Save parameters to JSON just in case
params_to_json(
    f"{params.file_label}/reconstruction_parameters_{params.file_label}.json"
)

## Verification

In [ ]:
charge_df.columns

In [ ]:
light_df.columns

In [ ]:
match_dict

In [ ]:
sipm_map

### Histograms

#### Charge

In [ ]:
print("Trigger time distribution")
charge_df["trig_time"].apply(np.mean).hist()

In [ ]:
print(f"Event duration in {params.time_unit}")
charge_df["event_duration"].hist()

In [ ]:
print(f"Charge per hit in {params.q_unit}")
(charge_df["event_q"] / charge_df["event_nhits"]).hist(bins=50)

In [ ]:
print(f"Charge per hit per event in {params.q_unit}")
(charge_df["event_q"] / charge_df["event_nhits"]).to_frame().reset_index().plot.scatter(
    x="eventID", y=0
)

In [ ]:
print(f"Event charge in {params.q_unit}")
charge_df["event_q"].hist()

In [ ]:
print(f"Hits q in {params.q_unit}")
charge_df["event_hits_q"].apply(tuple).explode().hist()

In [ ]:
print(f"Hits z in {params.z_unit}")
charge_df["event_hits_z"].apply(tuple).explode().hist()

#### Light

In [ ]:
light_df[params.light_variable].hist()

# Data fit

## Fake data map

In [ ]:
plot_fake_data([1], buffer=(params.xy_epsilon - 1))
if params.show_figures:
    plt.show()
else:
    plt.close("all")

## Main loop

In [ ]:
# Suppress the UndefinedMetricWarning
warnings.filterwarnings("ignore", category=Warning, module="sklearn")

In [ ]:
metrics = {}

if params.event_list is None:
    index_list = charge_df.index
else:
    index_list = charge_df.index.intersection(params.event_list)

light_indices = light_df["event"].copy()

for i, idx in enumerate(tqdm(index_list)):
    charge_values = pd.DataFrame(
        charge_df.loc[
            idx,
            [
                "event_hits_channelid",
                "event_hits_x",
                "event_hits_y",
                "event_hits_z",
                "event_hits_ts",
                "event_hits_q",
            ],
        ].to_list(),
        index=["ch", "x", "y", "z", "t", "q"],
    ).T

    non_zero_mask = (charge_values["ch"] != 0) * (
        charge_values["y"] != 0
    )  # Remove (0,0) entries
    noisy_channels_mask = ~charge_values["ch"].isin(
        [ch[0] for ch in params.channel_disable_list]
    )  # Disable channel 7
    mask = non_zero_mask * noisy_channels_mask  # Full hits mask

    # Apply boolean indexing to x, y, and z arrays
    charge_values = charge_values[mask]
    charge_values["q"] = charge_values["q"] * params.charge_gain  # Convert mV to ke

    # temp = index_list[i + 1] if i + 1 < len(index_list) else index_list[0]
    # light_event = match_dict.get(temp, [temp])[0]
    light_event = match_dict.get(idx, [idx])[0]
    light_matches = light_indices[light_indices == light_event].index
    # light_indices = light_indices[light_indices != light_event]
    light_values = light_df.loc[light_matches].dropna(subset=params.light_variable)

    if len(charge_values) > 2:
        if idx in params.individual_plots:
            metrics[idx] = event_display(
                idx,
                charge_values,
                light_values,
                plot_cyl=False,
            )
            if params.show_figures:
                plt.show()
            else:
                plt.close()
        else:
            # Create a design matrix
            labels = cluster_hits(charge_values[["x", "y", "z"]].to_numpy())
            # Fit clusters
            metrics[idx] = fit_hit_clusters(
                charge_values[["x", "y", "z"]].to_numpy(),
                charge_values["q"].to_numpy(),
                labels,
            )

        # Light to track geometry metrics
        track_lines = []
        for track_idx, values in metrics[idx].items():
            if "Fit_line" not in values:
                continue
            values["SiPM"] = light_geometry(
                track_line=values["Fit_line"],
                track_norm=values["Fit_norm"],
                sipm_df=light_values,
                light_variable=params.light_variable,
            )
            track_lines.append(values["Fit_line"])

        # Light and charge voxelization and fitting
        metrics[idx]["SiPM"] = voxelize_hits(
            charge_values,
            light_values,
            params.light_variable,
            charge_lines=track_lines,
        )

        metrics[idx][
            "Pixel_mask"
        ] = mask.to_numpy()  # Save masks to original dataframe for reference
        metrics[idx]["Total_light"] = light_values[params.light_variable].sum()
        metrics[idx]["Total_charge"] = charge_values["q"].sum()

In [ ]:
# Reset the warning filter (optional)
warnings.filterwarnings("default", category=Warning)

## Metrics

In [ ]:
# Save the metrics to a pickle file
metrics_file = f"{file_label}/metrics_{file_label}.pkl"
if params.overwrite_metrics or not os.path.isfile(metrics_file):
    with open(metrics_file, "wb") as f:
        pickle.dump(metrics, f)

    print(f"Metrics saved to {metrics_file}")

In [ ]:
metrics